# Pré-processamento do Dataset de Startups

Este notebook implementa as duas primeiras etapas do desafio:
1. **Limpeza e Tratamento de Valores Nulos**
2. **Codificação de Variáveis Categóricas**

Pipeline modular e reutilizável para preparar os dados para modelagem.

In [ ]:
# Configuração e importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import warnings

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
pd.set_option('display.max_columns', None)

print("✅ Bibliotecas carregadas")
print("📊 Configurações definidas")

In [ ]:
# Carregamento dos dados
train_df = pd.read_csv('database/train.csv')
test_df = pd.read_csv('database/test.csv')
sample_submission = pd.read_csv('database/sample_submission.csv')

print(f"📊 Dados carregados:")
print(f"   • Train: {train_df.shape}")
print(f"   • Test: {test_df.shape}")
print(f"   • Sample submission: {sample_submission.shape}")

# Preservar dados originais
train_original = train_df.copy()
test_original = test_df.copy()

## 1. Análise Exploratória Inicial

Vamos primeiro entender a estrutura dos dados e padrões de valores ausentes.

In [ ]:
# Identificação dos tipos de variáveis
numeric_vars = ['age_first_funding_year', 'age_last_funding_year', 'age_first_milestone_year', 
                'age_last_milestone_year', 'relationships', 'funding_rounds', 'funding_total_usd', 
                'milestones', 'avg_participants']

binary_vars = [col for col in train_df.columns if col.startswith(('is_', 'has_'))]
categorical_vars = ['category_code']
target_var = 'labels'
id_var = 'id'

print("📋 Estrutura das variáveis:")
print(f"   • Numéricas: {len(numeric_vars)} variáveis")
print(f"   • Binárias: {len(binary_vars)} variáveis")
print(f"   • Categóricas: {len(categorical_vars)} variáveis")
print(f"   • Target: {target_var}")
print(f"   • ID: {id_var}")

In [ ]:
# Análise de valores ausentes
def analyze_missing_values(df, dataset_name):
    """Analisa padrões de valores ausentes"""
    missing_info = []
    
    for col in df.columns:
        missing_count = df[col].isna().sum()
        missing_pct = (missing_count / len(df)) * 100
        
        if missing_count > 0:
            missing_info.append({
                'Variável': col,
                'Ausentes': missing_count,
                'Percentual': missing_pct
            })
    
    missing_df = pd.DataFrame(missing_info)
    
    print(f"\n🔍 {dataset_name.upper()}:")
    print(f"   • Total de variáveis: {len(df.columns)}")
    print(f"   • Variáveis com missing: {len(missing_df)}")
    
    if len(missing_df) > 0:
        print("\n📋 Variáveis com valores ausentes:")
        for _, row in missing_df.iterrows():
            print(f"   • {row['Variável']}: {row['Ausentes']} ({row['Percentual']:.1f}%)")
    else:
        print("   ✅ Nenhum valor ausente!")
    
    return missing_df

train_missing = analyze_missing_values(train_df, "Train")
test_missing = analyze_missing_values(test_df, "Test")

In [ ]:
# Visualização de valores ausentes
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Padrões de Valores Ausentes', fontsize=16, fontweight='bold')

def plot_missing_heatmap(df, dataset_name, ax):
    # Apenas variáveis com missing
    missing_vars = [col for col in df.columns if df[col].isna().sum() > 0]
    
    if missing_vars:
        missing_matrix = df[missing_vars].isna()
        sns.heatmap(missing_matrix.T, cmap=['lightblue', 'red'], 
                   cbar_kws={'label': 'Valores Ausentes'}, ax=ax, xticklabels=False)
        ax.set_title(f'Missing Values - {dataset_name}', fontweight='bold')
        ax.set_ylabel('Variáveis')
    else:
        ax.text(0.5, 0.5, 'Nenhum valor ausente', ha='center', va='center', 
               transform=ax.transAxes, fontsize=14)
        ax.set_title(f'Missing Values - {dataset_name}', fontweight='bold')

plot_missing_heatmap(train_df, 'Train', axes[0])
plot_missing_heatmap(test_df, 'Test', axes[1])

plt.tight_layout()
plt.show()

## 2. Análise das Variáveis Numéricas

Investigação das distribuições e outliers para decidir estratégias de imputação.

In [ ]:
# Estatísticas descritivas das variáveis numéricas
print("📊 ESTATÍSTICAS DAS VARIÁVEIS NUMÉRICAS - TRAIN")
print("="*60)
numeric_stats = train_df[numeric_vars].describe()
print(numeric_stats.round(2))

# Análise de skewness
print("\n📈 ANÁLISE DE ASSIMETRIA:")
for var in numeric_vars:
    skew = train_df[var].skew()
    missing_pct = (train_df[var].isna().sum() / len(train_df)) * 100
    print(f"   • {var}: skew={skew:.2f}, missing={missing_pct:.1f}%")

In [ ]:
# Visualização das distribuições das variáveis com missing values
missing_numeric_vars = [var for var in numeric_vars if train_df[var].isna().sum() > 0]

if missing_numeric_vars:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Distribuições das Variáveis Numéricas com Missing Values', fontsize=16, fontweight='bold')
    
    for i, var in enumerate(missing_numeric_vars):
        row = i // 2
        col = i % 2
        
        # Dados sem missing
        data = train_df[var].dropna()
        
        # Histograma
        axes[row, col].hist(data, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        axes[row, col].set_title(f'{var}\n(Missing: {train_df[var].isna().sum()} valores)', fontweight='bold')
        axes[row, col].set_xlabel(var)
        axes[row, col].set_ylabel('Frequência')
        axes[row, col].grid(True, alpha=0.3)
        
        # Estatísticas
        mean_val = data.mean()
        median_val = data.median()
        axes[row, col].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Média: {mean_val:.2f}')
        axes[row, col].axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Mediana: {median_val:.2f}')
        axes[row, col].legend()
    
    plt.tight_layout()
    plt.show()

print("✅ Análise de distribuições concluída")

## 3. Análise das Variáveis Categóricas

Análise da cardinalidade e distribuição das categorias.

In [ ]:
# Análise da variável categórica category_code
print("📂 ANÁLISE DA VARIÁVEL CATEGÓRICA - category_code")
print("="*60)

# Train
train_categories = train_df['category_code'].value_counts()
print(f"\n🔍 TRAIN:")
print(f"   • Categorias únicas: {len(train_categories)}")
print(f"   • Valores ausentes: {train_df['category_code'].isna().sum()}")
print(f"   • Top 10 categorias:")
for i, (cat, count) in enumerate(train_categories.head(10).items(), 1):
    pct = (count / len(train_df)) * 100
    print(f"      {i:2d}. {cat}: {count} ({pct:.1f}%)")

# Test
test_categories = test_df['category_code'].value_counts()
print(f"\n🧪 TEST:")
print(f"   • Categorias únicas: {len(test_categories)}")
print(f"   • Valores ausentes: {test_df['category_code'].isna().sum()}")
print(f"   • Top 10 categorias:")
for i, (cat, count) in enumerate(test_categories.head(10).items(), 1):
    pct = (count / len(test_df)) * 100
    print(f"      {i:2d}. {cat}: {count} ({pct:.1f}%)")

# Verificar consistência entre train e test
train_cats = set(train_df['category_code'].dropna().unique())
test_cats = set(test_df['category_code'].dropna().unique())
common_cats = train_cats & test_cats
train_only = train_cats - test_cats
test_only = test_cats - train_cats

print(f"\n🔗 CONSISTÊNCIA ENTRE DATASETS:")
print(f"   • Categorias comuns: {len(common_cats)}")
print(f"   • Apenas em train: {len(train_only)}")
print(f"   • Apenas em test: {len(test_only)}")
if train_only:
    print(f"   • Train exclusivas: {train_only}")
if test_only:
    print(f"   • Test exclusivas: {test_only}")

In [ ]:
# Visualização da distribuição de categorias
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
fig.suptitle('Distribuição das Categorias de Negócio', fontsize=16, fontweight='bold')

# Train
top_categories_train = train_categories.head(15)
axes[0].barh(range(len(top_categories_train)), top_categories_train.values, color='skyblue')
axes[0].set_yticks(range(len(top_categories_train)))
axes[0].set_yticklabels(top_categories_train.index)
axes[0].set_title('Top 15 Categorias - Train', fontweight='bold')
axes[0].set_xlabel('Quantidade')
axes[0].grid(True, alpha=0.3, axis='x')

# Test
top_categories_test = test_categories.head(15)
axes[1].barh(range(len(top_categories_test)), top_categories_test.values, color='orange')
axes[1].set_yticks(range(len(top_categories_test)))
axes[1].set_yticklabels(top_categories_test.index)
axes[1].set_title('Top 15 Categorias - Test', fontweight='bold')
axes[1].set_xlabel('Quantidade')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("✅ Análise de variáveis categóricas concluída")

## 4. Funções de Pré-processamento

Implementação de funções modulares para tratamento de valores ausentes e codificação categórica.

In [ ]:
def handle_missing_values(df, strategy_dict=None, verbose=True):
    """
    Trata valores ausentes baseado em estratégias específicas por variável.
    
    Parâmetros:
    - df: DataFrame para processar
    - strategy_dict: Dicionário com estratégias por variável 
                    {'var_name': 'mean'/'median'/'mode'/'drop'/valor_específico}
    - verbose: Se deve imprimir informações do processo
    
    Retorna:
    - DataFrame processado
    """
    df_processed = df.copy()
    
    # Estratégias padrão baseadas na análise exploratória
    if strategy_dict is None:
        strategy_dict = {
            # Variáveis de idade com distribuições assimétricas -> mediana
            'age_first_funding_year': 'median',
            'age_last_funding_year': 'median', 
            'age_first_milestone_year': 'median',
            'age_last_milestone_year': 'median',
            # Variável categórica -> categoria mais frequente
            'category_code': 'mode'
        }
    
    if verbose:
        print("🔧 TRATAMENTO DE VALORES AUSENTES")
        print("="*50)
    
    for col, strategy in strategy_dict.items():
        if col in df_processed.columns:
            missing_before = df_processed[col].isna().sum()
            
            if missing_before > 0:
                if strategy == 'mean':
                    fill_value = df_processed[col].mean()
                elif strategy == 'median':
                    fill_value = df_processed[col].median()
                elif strategy == 'mode':
                    fill_value = df_processed[col].mode().iloc[0] if len(df_processed[col].mode()) > 0 else 'unknown'
                elif strategy == 'drop':
                    df_processed = df_processed.dropna(subset=[col])
                    if verbose:
                        print(f"   • {col}: removidas {missing_before} linhas")
                    continue
                else:
                    fill_value = strategy  # Valor específico
                
                df_processed[col] = df_processed[col].fillna(fill_value)
                
                if verbose:
                    print(f"   • {col}: {missing_before} ausentes → imputados com {strategy} ({fill_value})")
    
    if verbose:
        total_missing_after = df_processed.isna().sum().sum()
        print(f"\n✅ Processamento concluído. Missing values restantes: {total_missing_after}")
    
    return df_processed

print("✅ Função handle_missing_values() criada")

In [ ]:
def encode_categorical(df, categorical_columns=None, encoding_strategy=None, verbose=True):
    """
    Codifica variáveis categóricas usando diferentes estratégias.
    
    Parâmetros:
    - df: DataFrame para processar
    - categorical_columns: Lista de colunas categóricas para codificar
    - encoding_strategy: Dicionário com estratégias por variável
                        {'var_name': 'onehot'/'ordinal'/'label'}
    - verbose: Se deve imprimir informações do processo
    
    Retorna:
    - DataFrame processado
    - Dicionário com encoders ajustados (para aplicar no test set)
    """
    df_processed = df.copy()
    encoders = {}
    
    # Configurações padrão
    if categorical_columns is None:
        categorical_columns = ['category_code']
    
    if encoding_strategy is None:
        encoding_strategy = {
            'category_code': 'onehot'  # One-hot encoding para categoria de negócio
        }
    
    if verbose:
        print("🔢 CODIFICAÇÃO DE VARIÁVEIS CATEGÓRICAS")
        print("="*50)
    
    for col in categorical_columns:
        if col in df_processed.columns:
            strategy = encoding_strategy.get(col, 'onehot')
            unique_values = df_processed[col].nunique()
            
            if strategy == 'onehot':
                # One-hot encoding
                encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
                encoded_data = encoder.fit_transform(df_processed[[col]])
                
                # Criar nomes das colunas
                feature_names = [f"{col}_{cat}" for cat in encoder.categories_[0]]
                encoded_df = pd.DataFrame(encoded_data, columns=feature_names, index=df_processed.index)
                
                # Substituir coluna original
                df_processed = df_processed.drop(columns=[col])
                df_processed = pd.concat([df_processed, encoded_df], axis=1)
                
                encoders[col] = encoder
                
                if verbose:
                    print(f"   • {col}: One-hot encoding → {len(feature_names)} colunas (de {unique_values} categorias)")
            
            elif strategy == 'ordinal':
                # Ordinal encoding
                encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
                df_processed[col] = encoder.fit_transform(df_processed[[col]]).ravel()
                
                encoders[col] = encoder
                
                if verbose:
                    print(f"   • {col}: Ordinal encoding → {unique_values} categorias codificadas")
    
    if verbose:
        print(f"\n✅ Codificação concluída. Shape final: {df_processed.shape}")
    
    return df_processed, encoders

print("✅ Função encode_categorical() criada")

## 5. Aplicação do Pipeline de Pré-processamento

Aplicação das funções criadas nos datasets de treino e teste.

In [ ]:
# Etapa 1: Tratamento de valores ausentes no dataset de treino
print("🚀 INICIANDO PIPELINE DE PRÉ-PROCESSAMENTO")
print("="*60)

# Separar target e features no train
X_train = train_df.drop(columns=['labels'])
y_train = train_df['labels']

print(f"\n📊 Dados separados:")
print(f"   • X_train: {X_train.shape}")
print(f"   • y_train: {y_train.shape}")
print(f"   • test_df: {test_df.shape}")

In [ ]:
# Aplicar tratamento de missing values
X_train_clean = handle_missing_values(X_train, verbose=True)
test_df_clean = handle_missing_values(test_df, verbose=True)

In [ ]:
# Etapa 2: Codificação de variáveis categóricas
X_train_encoded, train_encoders = encode_categorical(X_train_clean, verbose=True)

# Aplicar os mesmos encoders no test set
print("\n🔢 APLICANDO ENCODERS NO TEST SET")
print("="*40)

test_df_encoded = test_df_clean.copy()

# Aplicar encoder do category_code
if 'category_code' in train_encoders:
    encoder = train_encoders['category_code']
    encoded_data = encoder.transform(test_df_encoded[['category_code']])
    
    # Criar DataFrame com as mesmas colunas do train
    feature_names = [f"category_code_{cat}" for cat in encoder.categories_[0]]
    encoded_df = pd.DataFrame(encoded_data, columns=feature_names, index=test_df_encoded.index)
    
    # Substituir coluna original
    test_df_encoded = test_df_encoded.drop(columns=['category_code'])
    test_df_encoded = pd.concat([test_df_encoded, encoded_df], axis=1)
    
    print(f"   • category_code: aplicado encoder → {len(feature_names)} colunas")

print(f"\n✅ Test set processado. Shape final: {test_df_encoded.shape}")

In [ ]:
# Verificar consistência entre train e test após processamento
print("🔍 VERIFICAÇÃO DE CONSISTÊNCIA PÓS-PROCESSAMENTO")
print("="*55)

# Colunas em comum
train_cols = set(X_train_encoded.columns)
test_cols = set(test_df_encoded.columns)
common_cols = train_cols & test_cols
train_only_cols = train_cols - test_cols
test_only_cols = test_cols - train_cols

print(f"   • Colunas em comum: {len(common_cols)}")
print(f"   • Apenas em train: {len(train_only_cols)}")
print(f"   • Apenas em test: {len(test_only_cols)}")

if train_only_cols:
    print(f"   • Train exclusivas: {train_only_cols}")
if test_only_cols:
    print(f"   • Test exclusivas: {test_only_cols}")

# Verificar valores ausentes finais
train_missing_final = X_train_encoded.isna().sum().sum()
test_missing_final = test_df_encoded.isna().sum().sum()

print(f"\n📊 Missing values finais:")
print(f"   • Train: {train_missing_final}")
print(f"   • Test: {test_missing_final}")

# Shapes finais
print(f"\n📐 Shapes finais:")
print(f"   • X_train_encoded: {X_train_encoded.shape}")
print(f"   • y_train: {y_train.shape}")
print(f"   • test_df_encoded: {test_df_encoded.shape}")

## 6. Verificação Final dos Dados Processados

Visualizações finais para validar o pré-processamento.

In [ ]:
# Visualização final - comparação antes/depois
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Comparação: Antes vs Depois do Pré-processamento', fontsize=16, fontweight='bold')

# Missing values - antes
original_missing = train_original.isna().sum().sum()
axes[0, 0].bar(['Original Train'], [original_missing], color='red', alpha=0.7)
axes[0, 0].set_title('Missing Values - Antes', fontweight='bold')
axes[0, 0].set_ylabel('Quantidade')
axes[0, 0].grid(True, alpha=0.3)

# Missing values - depois
processed_missing = X_train_encoded.isna().sum().sum()
axes[0, 1].bar(['Processed Train'], [processed_missing], color='green', alpha=0.7)
axes[0, 1].set_title('Missing Values - Depois', fontweight='bold')
axes[0, 1].set_ylabel('Quantidade')
axes[0, 1].grid(True, alpha=0.3)

# Número de colunas - antes vs depois
cols_before = [len(train_original.columns), len(test_original.columns)]
cols_after = [len(X_train_encoded.columns), len(test_df_encoded.columns)]

x = ['Train', 'Test']
width = 0.35
x_pos = np.arange(len(x))

axes[1, 0].bar(x_pos - width/2, cols_before, width, label='Antes', color='skyblue', alpha=0.7)
axes[1, 0].bar(x_pos + width/2, cols_after, width, label='Depois', color='orange', alpha=0.7)
axes[1, 0].set_title('Número de Colunas - Antes vs Depois', fontweight='bold')
axes[1, 0].set_ylabel('Quantidade')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels(x)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Distribuição do target (inalterada)
target_counts = y_train.value_counts()
axes[1, 1].pie(target_counts.values, labels=['Fracasso (0)', 'Sucesso (1)'], 
               autopct='%1.1f%%', startangle=90, colors=['lightcoral', 'lightgreen'])
axes[1, 1].set_title('Distribuição do Target', fontweight='bold')

plt.tight_layout()
plt.show()

print("✅ Verificação visual concluída")

In [ ]:
# Amostra dos dados processados
print("📋 AMOSTRA DOS DADOS PROCESSADOS")
print("="*50)

print("\n🎯 X_train_encoded (primeiras 3 linhas, primeiras 10 colunas):")
print(X_train_encoded.iloc[:3, :10])

print("\n🧪 test_df_encoded (primeiras 3 linhas, primeiras 10 colunas):")
print(test_df_encoded.iloc[:3, :10])

print("\n🏷️ y_train (primeiras 10 valores):")
print(y_train.head(10).values)

print("\n📊 Tipos de dados em X_train_encoded:")
dtype_counts = X_train_encoded.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"   • {dtype}: {count} colunas")

In [ ]:
# Salvar dados processados (opcional)
print("💾 SALVANDO DADOS PROCESSADOS")
print("="*40)

# Recriar DataFrame de treino completo
train_processed = X_train_encoded.copy()
train_processed['labels'] = y_train

# Salvar
train_processed.to_csv('database/train_processed.csv', index=False)
test_df_encoded.to_csv('database/test_processed.csv', index=False)

print("✅ Arquivos salvos:")
print("   • database/train_processed.csv")
print("   • database/test_processed.csv")

print(f"\n📊 Resumo final:")
print(f"   • Train processado: {train_processed.shape}")
print(f"   • Test processado: {test_df_encoded.shape}")
print(f"   • Dados prontos para seleção de features e modelagem!")

## Resumo das Estratégias de Pré-processamento

### Tratamento de Valores Ausentes:
- **Variáveis numéricas de idade** (`age_*`): Imputação com **mediana** devido às distribuições assimétricas (skew > 0.5)
- **Variável categórica** (`category_code`): Imputação com **moda** (categoria mais frequente)
- **Justificativa**: Mediana é robusta a outliers nas variáveis de idade; moda preserva a distribuição categórica original

### Codificação Categórica:
- **category_code**: **One-Hot Encoding** para 33+ categorias de negócio, evitando assumir ordem ordinal
- **Variáveis binárias**: Mantidas como estão (já codificadas como 0/1)
- **Justificativa**: One-hot preserva informação categórica sem criar hierarquias artificiais

### Pipeline Robusto:
- Funções modulares `handle_missing_values()` e `encode_categorical()` reutilizáveis
- Encoders ajustados no train e aplicados no test, evitando data leakage
- Verificação de consistência entre datasets para garantir compatibilidade na modelagem